# Freshness Predictor - Model Training

This notebook fine-tunes a ResNet model for regression to predict days remaining until spoiled.

## Dataset Structure
Your dataset should be organized as follows:
```
dataset/
  day_0/
    image1.jpg
    image2.jpg
  day_1/
    image1.jpg
    ...
  day_5/
    image1.jpg
    ...
```

Each folder name represents the number of days remaining until spoiled.


In [ ]:
# Install required packages
!pip install tensorflow pillow numpy


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from pathlib import Path

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")


In [ ]:
# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0001
DATASET_PATH = "/content/dataset"  # Update this path
MODEL_SAVE_PATH = "/content/model.h5"

# Number of days (0-5)
NUM_CLASSES = 6  # 0, 1, 2, 3, 4, 5 days


In [ ]:
# Load and prepare dataset
def load_dataset_from_folders(dataset_path):
    """
    Load images from folder structure where folder names are day labels.
    Returns: (images, labels) where labels are numeric (0-5)
    """
    images = []
    labels = []
    
    dataset_path = Path(dataset_path)
    
    for day_folder in sorted(dataset_path.iterdir()):
        if not day_folder.is_dir():
            continue
            
        # Extract day number from folder name (e.g., 'day_0' -> 0)
        try:
            day_label = int(day_folder.name.split('_')[1])
        except:
            print(f"Skipping folder: {day_folder.name}")
            continue
        
        # Load all images in this folder
        for img_file in list(day_folder.glob('*.jpg')) + list(day_folder.glob('*.png')):
            img = keras.preprocessing.image.load_img(
                img_file, target_size=IMG_SIZE
            )
            img_array = keras.preprocessing.image.img_to_array(img)
            images.append(img_array)
            labels.append(float(day_label))
    
    return np.array(images), np.array(labels)

# Load dataset
print("Loading dataset...")
images, labels = load_dataset_from_folders(DATASET_PATH)
print(f"Loaded {len(images)} images with labels shape: {labels.shape}")
print(f"Label range: {labels.min()} to {labels.max()}")


In [ ]:
# Normalize images
images = images / 255.0

# Split into train and validation sets (80/20)
split_idx = int(len(images) * 0.8)
train_images = images[:split_idx]
train_labels = labels[:split_idx]
val_images = images[split_idx:]
val_labels = labels[split_idx:]

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")


In [ ]:
# Build the model
# Load pre-trained ResNet50 (without top classification layer)
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMG_SIZE, 3)
)

# Freeze base model initially
base_model.trainable = False

# Add custom regression head
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation=None)  # Regression output (no activation)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='mse',  # Mean Squared Error for regression
    metrics=['mae']  # Mean Absolute Error
)

model.summary()


In [ ]:
# Train the model (Phase 1: Frozen base)
print("Phase 1: Training with frozen base model...")

history1 = model.fit(
    train_images, train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS // 2,
    validation_data=(val_images, val_labels),
    verbose=1
)
